# 🚀 BrainX - Sistema Completo Adaptativo

## ⚠️ IMPORTANTE: Este notebook foi otimizado para Google Colab com GPU

**Para melhor performance:**
1. Faça upload deste notebook no Google Colab: https://colab.research.google.com
2. Habilite GPU: Runtime → Change runtime type → GPU
3. Execute todas as células na ordem

**Funciona localmente também**, mas será mais lento sem GPU.

---

Sistema completo com todas as melhorias para superar todos os modelos:
- ✅ Prompts adaptativos por TRI
- ✅ Few-shots customizados por tema
- ✅ Detecção de figuras simples
- ✅ Self-Consistency (múltiplas passagens)
- ✅ Prompt ultra-simples para questões fáceis
- ✅ Prompt especializado para Natureza

**Meta**: 94%+ acurácia (superar GPT-4o com 93.85%)


## 📦 Instalação e Configuração


In [44]:
# Instalar dependências
try:
    get_ipython().run_line_magic('pip', 'install -q openai python-dotenv tqdm')
except:
    import subprocess
    subprocess.run(['pip', 'install', '-q', 'openai', 'python-dotenv', 'tqdm'])

# Verificar GPU e ambiente
print("🔍 VERIFICANDO AMBIENTE E GPU")
print("=" * 60)

# Verificar se está no Colab
try:
    import google.colab
    print("✅ Google Colab detectado")
    IN_COLAB = True
    print("💡 Para habilitar GPU: Runtime → Change runtime type → GPU")
except ImportError:
    print("⚠️  Executando LOCALMENTE (não no Google Colab)")
    print("   💡 Para melhor performance, use Google Colab com GPU")
    print("   📌 Link: https://colab.research.google.com")
    IN_COLAB = False

# Verificar GPU
USANDO_GPU = False
try:
    import torch
    if torch.cuda.is_available():
        device_name = torch.cuda.get_device_name(0)
        memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"\n✅ GPU disponível: {device_name}")
        print(f"   Memória GPU: {memory_gb:.1f} GB")
        print(f"   CUDA Version: {torch.version.cuda}")
        USANDO_GPU = True
    else:
        print(f"\n⚠️  GPU não disponível (usando CPU)")
        if IN_COLAB:
            print("   💡 Dica: Vá em Runtime → Change runtime type → GPU")
        else:
            print("   💡 Para GPU, execute no Google Colab")
except ImportError:
    print("\n⚠️  PyTorch não instalado")
    if IN_COLAB:
        try:
            get_ipython().run_line_magic('pip', 'install -q torch')
        except:
            pass
        try:
            import torch
            if torch.cuda.is_available():
                print(f"✅ GPU disponível: {torch.cuda.get_device_name(0)}")
                USANDO_GPU = True
            else:
                print("⚠️  GPU não disponível")
        except:
            pass
    else:
        print("   (PyTorch opcional para verificação local)")

# Verificar variáveis de ambiente
import os
if IN_COLAB:
    print(f"\n📊 Informações do Colab:")
    print(f"   COLAB_GPU: {os.environ.get('COLAB_GPU', 'não definida')}")

print(f"\n💡 Nota sobre Performance:")
if IN_COLAB and USANDO_GPU:
    print(f"   ✅ Ambiente otimizado: Colab + GPU")
elif IN_COLAB:
    print(f"   ⚠️  Colab sem GPU - habilite GPU para melhor performance")
else:
    print(f"   ⚠️  Executando localmente - será mais lento")
    print(f"   💡 Recomendado: Use Google Colab com GPU para processar mais rápido")
print(f"   - Chamadas API são via rede (não usam GPU diretamente)")
print(f"   - Mas GPU permite otimizações e processamento paralelo")


Note: you may need to restart the kernel to use updated packages.
🔍 VERIFICANDO AMBIENTE E GPU
⚠️  Executando LOCALMENTE (não no Google Colab)
   💡 Para melhor performance, use Google Colab com GPU
   📌 Link: https://colab.research.google.com

⚠️  GPU não disponível (usando CPU)
   💡 Para GPU, execute no Google Colab

💡 Nota sobre Performance:
   ⚠️  Executando localmente - será mais lento
   💡 Recomendado: Use Google Colab com GPU para processar mais rápido
   - Chamadas API são via rede (não usam GPU diretamente)
   - Mas GPU permite otimizações e processamento paralelo


In [45]:
# Clonar repositório (com detecção de caminho duplicado)
import os
from pathlib import Path
import subprocess
import shutil

current_dir = Path.cwd()
print(f"📁 Diretório inicial: {current_dir}")

# DETECTAR E CORRIGIR CAMINHO DUPLICADO
# Se o caminho contém "gpt-4-enem/gpt-4-enem", estamos em um diretório duplicado
path_str = str(current_dir)
if path_str.count('gpt-4-enem') >= 2:
    # Encontrar o primeiro "gpt-4-enem" válido (que tem scripts/analise_enem)
    parts = path_str.split('/')
    for i in range(len(parts)):
        if parts[i] == 'gpt-4-enem':
            # Tentar caminho até este ponto
            test_path = Path('/'.join(parts[:i+1]))
            if (test_path / 'scripts' / 'analise_enem').exists():
                print(f"🔧 Detectado caminho duplicado! Corrigindo...")
                print(f"   Caminho duplicado: {current_dir}")
                print(f"   Caminho correto: {test_path}")
                os.chdir(test_path)
                current_dir = Path.cwd()
                print(f"✅ Corrigido! Agora em: {current_dir}")
                break

# Estratégia: encontrar o diretório raiz do repositório
repo_root = None

# Verificar se já estamos no diretório raiz do repositório
if (current_dir / 'scripts' / 'analise_enem').exists():
    repo_root = current_dir
    print(f"✅ Já estamos no diretório raiz do repositório: {repo_root}")
elif (current_dir.parent / 'scripts' / 'analise_enem').exists():
    repo_root = current_dir.parent
    print(f"✅ Repositório encontrado no diretório pai: {repo_root}")
elif (current_dir.parent.parent / 'scripts' / 'analise_enem').exists():
    repo_root = current_dir.parent.parent
    print(f"✅ Repositório encontrado 2 níveis acima: {repo_root}")

# Se ainda não encontrou, verificar se precisa clonar
if repo_root is None:
    # Verificar se o diretório atual é "gpt-4-enem" ou está dentro dele
    if current_dir.name == 'gpt-4-enem' and (current_dir / 'scripts' / 'analise_enem').exists():
        repo_root = current_dir
        print(f"✅ Já estamos dentro do repositório: {repo_root}")
    elif (current_dir / 'gpt-4-enem' / 'scripts' / 'analise_enem').exists():
        repo_root = current_dir / 'gpt-4-enem'
        print(f"✅ Repositório encontrado em subdiretório: {repo_root}")
    else:
        # Determinar onde clonar baseado no diretório atual
        # Se estamos em notebooks/, ir para a raiz do Colab (geralmente /content)
        if 'notebooks' in str(current_dir) or current_dir.name == 'notebooks':
            # Ir para a raiz do Colab (geralmente /content)
            colab_root = Path('/content') if Path('/content').exists() else current_dir
            while colab_root.parent != colab_root:  # Enquanto não for a raiz
                if (colab_root / 'scripts' / 'analise_enem').exists():
                    repo_root = colab_root
                    break
                if colab_root.parent == colab_root:
                    break
                colab_root = colab_root.parent
            
            if repo_root is None:
                clone_target = colab_root / 'gpt-4-enem'
                if (clone_target / 'scripts' / 'analise_enem').exists():
                    repo_root = clone_target
                    print(f"✅ Repositório já existe: {repo_root}")
                elif not clone_target.exists():
                    print(f"📥 Clonando repositório em: {clone_target.parent}")
                    subprocess.run(['git', 'clone', 'https://github.com/xtribr/enemnlp.git', str(clone_target)], 
                                 cwd=str(clone_target.parent), check=True)
                    repo_root = clone_target
                    print(f"✅ Repositório clonado em: {repo_root}")
                else:
                    repo_root = clone_target
        else:
            # Clonar no diretório atual
            clone_target = current_dir / 'gpt-4-enem'
            if not clone_target.exists():
                print(f"📥 Clonando repositório...")
                subprocess.run(['git', 'clone', 'https://github.com/xtribr/enemnlp.git', 'gpt-4-enem'], 
                             cwd=str(current_dir), check=True)
                repo_root = clone_target
                print(f"✅ Repositório clonado em: {repo_root}")
            elif (clone_target / 'scripts' / 'analise_enem').exists():
                repo_root = clone_target
                print(f"✅ Repositório já existe: {repo_root}")
            else:
                repo_root = clone_target

# Mudar para o diretório raiz do repositório
if repo_root and repo_root.exists():
    os.chdir(repo_root)
    final_dir = Path.cwd()
    print(f"📁 Diretório final: {final_dir}")
    
    # Verificar se está correto
    if (final_dir / 'scripts' / 'analise_enem').exists():
        print("✅ Diretório correto! scripts/analise_enem encontrado")
    else:
        print(f"⚠️  Atenção: scripts/analise_enem não encontrado em {final_dir}")
        print("   A célula seguinte tentará encontrar automaticamente")
else:
    print("❌ Não foi possível encontrar ou criar o repositório")
    print(f"   Diretório atual permanece: {Path.cwd()}")


📁 Diretório inicial: /Users/bunker/gpt-4-enem/gpt-4-enem
🔧 Detectado caminho duplicado! Corrigindo...
   Caminho duplicado: /Users/bunker/gpt-4-enem/gpt-4-enem
   Caminho correto: /Users/bunker/gpt-4-enem/gpt-4-enem
✅ Corrigido! Agora em: /Users/bunker/gpt-4-enem/gpt-4-enem
✅ Já estamos no diretório raiz do repositório: /Users/bunker/gpt-4-enem/gpt-4-enem
📁 Diretório final: /Users/bunker/gpt-4-enem/gpt-4-enem
✅ Diretório correto! scripts/analise_enem encontrado


In [46]:
# Configurar API Key da Maritaca
from getpass import getpass
import os
from pathlib import Path
from dotenv import load_dotenv

# Tentar carregar do .env se existir
env_file = Path('.env')
if env_file.exists():
    load_dotenv(env_file)
    print("✅ .env carregado")

# Se não estiver configurada, pedir ao usuário
if not os.getenv('CURSORMINIMAC') and not os.getenv('MARITALK_API_SECRET_KEY'):
    print("🔑 Configure sua chave API da Maritaca:")
    api_key = getpass("Cole sua chave (será ocultada): ")
    os.environ['CURSORMINIMAC'] = api_key
    print("✅ Chave configurada!")
else:
    api_key = os.getenv('CURSORMINIMAC') or os.getenv('MARITALK_API_SECRET_KEY')
    print(f"✅ Chave já configurada: ...{api_key[-10:] if api_key else 'N/A'}")


✅ .env carregado
✅ Chave já configurada: ...697ebc2587


## 🧠 Importar Sistema BrainX


In [47]:
import sys
import json
import time
from collections import Counter, defaultdict
from pathlib import Path
from typing import Dict, List, Optional
import importlib.util
import os

# Verificar diretório atual e corrigir se necessário
current_dir = Path.cwd()
print(f"📁 Diretório atual: {current_dir}")

# Verificar diretório atual
current_dir = Path.cwd()
print(f"📁 Diretório atual: {current_dir}")

# Verificar se scripts/analise_enem existe no diretório atual
scripts_dir = current_dir / 'scripts' / 'analise_enem'

if not scripts_dir.exists():
    # Se não existe, procurar em locais alternativos
    print("🔍 Procurando scripts/analise_enem em locais alternativos...")
    
    possible_paths = [
        current_dir / 'scripts' / 'analise_enem',
        current_dir / 'gpt-4-enem' / 'scripts' / 'analise_enem',
        current_dir.parent / 'scripts' / 'analise_enem',
        current_dir.parent.parent / 'scripts' / 'analise_enem',
        Path('/content') / 'gpt-4-enem' / 'scripts' / 'analise_enem',  # Colab padrão
        Path('/content') / 'enemnlp' / 'scripts' / 'analise_enem',  # Se clonou como enemnlp
    ]
    
    found_path = None
    for path in possible_paths:
        if path.exists():
            found_path = path
            # Mudar para o diretório raiz (2 níveis acima de scripts/analise_enem)
            project_root = path.parent.parent
            os.chdir(project_root)
            scripts_dir = Path('scripts/analise_enem')
            print(f"✅ Encontrado em: {found_path}")
            print(f"📁 Mudando para diretório raiz: {Path.cwd()}")
            break
    
    if found_path is None:
        print(f"❌ Diretório scripts/analise_enem não encontrado!")
        print(f"   Diretório atual: {Path.cwd()}")
        print(f"   Tentamos os seguintes caminhos:")
        for path in possible_paths:
            exists = path.exists()
            print(f"     - {path} {'✅' if exists else '❌'}")
        print(f"\n   Conteúdo do diretório atual:")
        try:
            items = list(Path('.').iterdir())
            for item in items[:15]:  # Mostrar mais itens
                print(f"     - {item.name} {'(dir)' if item.is_dir() else ''}")
            if len(items) > 15:
                print(f"     ... e mais {len(items) - 15} itens")
        except Exception as e:
            print(f"     Erro ao listar: {e}")
        print(f"\n💡 Solução: Certifique-se de que o repositório foi clonado completamente")
        raise FileNotFoundError(f"Diretório scripts/analise_enem não encontrado. Verifique se o repositório foi clonado corretamente.")

# Verificar novamente após possível mudança de diretório
if not scripts_dir.exists():
    scripts_dir = Path('scripts/analise_enem')
    
if not scripts_dir.exists():
    raise FileNotFoundError(f"scripts/analise_enem não existe em {Path.cwd()}")

# Verificar se arquivos essenciais existem
arquivos_essenciais = [
    '70_prompts_adaptativos_por_tri.py',
    '75_deteccao_figuras_simples.py',
    '79_prompt_ultra_simples_facil.py'
]

print(f"✅ Usando scripts em: {scripts_dir.absolute()}")
print(f"\n🔍 Verificando arquivos essenciais:")
arquivos_faltando = []
for arquivo in arquivos_essenciais:
    arquivo_path = scripts_dir / arquivo
    if arquivo_path.exists():
        print(f"   ✅ {arquivo}")
    else:
        print(f"   ❌ {arquivo} - NÃO ENCONTRADO!")
        arquivos_faltando.append(arquivo)

if arquivos_faltando:
    print(f"\n⚠️  Arquivos faltando: {', '.join(arquivos_faltando)}")
    print(f"   Isso pode indicar que o repositório não foi clonado completamente")
    print(f"   Tente clonar novamente ou verifique se os arquivos existem no GitHub")

# Adicionar scripts ao path
sys.path.insert(0, str(scripts_dir.absolute()))
sys.path.insert(0, str(Path('.').absolute()))

# Importar módulos do BrainX
# Prompts adaptativos
prompts_path = scripts_dir / '70_prompts_adaptativos_por_tri.py'
if not prompts_path.exists():
    print(f"❌ ERRO: Arquivo não encontrado: {prompts_path.absolute()}")
    print(f"\n🔧 Tentando soluções alternativas...")
    
    # Tentar baixar do GitHub se não existir
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/70_prompts_adaptativos_por_tri.py"
        print(f"   📥 Baixando do GitHub: {github_url}")
        urllib.request.urlretrieve(github_url, str(prompts_path))
        print(f"   ✅ Arquivo baixado com sucesso!")
    except Exception as e:
        print(f"   ❌ Erro ao baixar: {str(e)[:100]}")
        raise FileNotFoundError(f"Arquivo não encontrado e não foi possível baixar: {prompts_path.absolute()}")

spec = importlib.util.spec_from_file_location('prompts_adaptativos', str(prompts_path.absolute()))
prompts_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(prompts_module)
print("✅ Prompts adaptativos carregados")

# Few-shots
fewshots_path = scripts_dir / '73_fewshots_customizados_por_tema.py'
if not fewshots_path.exists():
    print(f"⚠️  Arquivo não encontrado, tentando baixar do GitHub...")
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/73_fewshots_customizados_por_tema.py"
        urllib.request.urlretrieve(github_url, str(fewshots_path))
        print(f"   ✅ Arquivo baixado!")
    except Exception as e:
        print(f"   ⚠️  Não foi possível baixar: {str(e)[:100]}")
        fewshots_module = None
        print("⚠️  Few-shots customizados não encontrado (opcional)")
else:
    spec2 = importlib.util.spec_from_file_location('fewshots_customizados', str(fewshots_path.absolute()))
    fewshots_module = importlib.util.module_from_spec(spec2)
    spec2.loader.exec_module(fewshots_module)
    print("✅ Few-shots customizados carregados")

# Detecção de figuras
figuras_path = scripts_dir / '75_deteccao_figuras_simples.py'
if not figuras_path.exists():
    print(f"⚠️  Arquivo não encontrado, tentando baixar do GitHub...")
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/75_deteccao_figuras_simples.py"
        urllib.request.urlretrieve(github_url, str(figuras_path))
        print(f"   ✅ Arquivo baixado!")
    except Exception as e:
        raise FileNotFoundError(f"Arquivo não encontrado: {figuras_path.absolute()}")
spec3 = importlib.util.spec_from_file_location('deteccao_figuras', str(figuras_path.absolute()))
figuras_module = importlib.util.module_from_spec(spec3)
spec3.loader.exec_module(figuras_module)
print("✅ Detecção de figuras carregada")

# Prompt simples
simples_path = scripts_dir / '79_prompt_ultra_simples_facil.py'
if not simples_path.exists():
    print(f"⚠️  Arquivo não encontrado, tentando baixar do GitHub...")
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/79_prompt_ultra_simples_facil.py"
        urllib.request.urlretrieve(github_url, str(simples_path))
        print(f"   ✅ Arquivo baixado!")
    except Exception as e:
        raise FileNotFoundError(f"Arquivo não encontrado: {simples_path.absolute()}")
spec4 = importlib.util.spec_from_file_location('prompt_simples', str(simples_path.absolute()))
simples_module = importlib.util.module_from_spec(spec4)
spec4.loader.exec_module(simples_module)
print("✅ Prompt ultra-simples carregado")

# Few-shots Natureza
natureza_path = scripts_dir / '81_fewshots_natureza_expandido.py'
if not natureza_path.exists():
    print(f"⚠️  Arquivo não encontrado, tentando baixar do GitHub...")
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/81_fewshots_natureza_expandido.py"
        urllib.request.urlretrieve(github_url, str(natureza_path))
        print(f"   ✅ Arquivo baixado!")
    except Exception as e:
        print(f"   ⚠️  Não foi possível baixar: {str(e)[:100]}")
        natureza_module = None
        print("⚠️  Few-shots Natureza não encontrado (opcional)")
else:
    spec5 = importlib.util.spec_from_file_location('fewshots_natureza', str(natureza_path.absolute()))
    natureza_module = importlib.util.module_from_spec(spec5)
    spec5.loader.exec_module(natureza_module)
    print("✅ Few-shots Natureza carregados")

# Prompt Natureza
prompt_nat_path = scripts_dir / '82_prompt_especializado_natureza.py'
if not prompt_nat_path.exists():
    print(f"⚠️  Arquivo não encontrado, tentando baixar do GitHub...")
    try:
        import urllib.request
        github_url = "https://raw.githubusercontent.com/xtribr/enemnlp/main/scripts/analise_enem/82_prompt_especializado_natureza.py"
        urllib.request.urlretrieve(github_url, str(prompt_nat_path))
        print(f"   ✅ Arquivo baixado!")
    except Exception as e:
        print(f"   ⚠️  Não foi possível baixar: {str(e)[:100]}")
        prompt_nat_module = None
        print("⚠️  Prompt Natureza não encontrado (opcional)")
else:
    spec6 = importlib.util.spec_from_file_location('prompt_natureza', str(prompt_nat_path.absolute()))
    prompt_nat_module = importlib.util.module_from_spec(spec6)
    spec6.loader.exec_module(prompt_nat_module)
    print("✅ Prompt Natureza carregado")

print("\n✅ Todos os módulos do BrainX carregados!")


📁 Diretório atual: /Users/bunker/gpt-4-enem/gpt-4-enem
📁 Diretório atual: /Users/bunker/gpt-4-enem/gpt-4-enem
✅ Usando scripts em: /Users/bunker/gpt-4-enem/gpt-4-enem/scripts/analise_enem

🔍 Verificando arquivos essenciais:
   ✅ 70_prompts_adaptativos_por_tri.py
   ✅ 75_deteccao_figuras_simples.py
   ✅ 79_prompt_ultra_simples_facil.py
✅ Prompts adaptativos carregados
✅ Few-shots customizados carregados
✅ Detecção de figuras carregada
✅ Prompt ultra-simples carregado
✅ Few-shots Natureza carregados
✅ Prompt Natureza carregado

✅ Todos os módulos do BrainX carregados!


In [48]:
import openai
from tqdm import tqdm

def configurar_api_colab():
    """Configura API para Colab"""
    api_key = os.getenv('CURSORMINIMAC') or os.getenv('MARITALK_API_SECRET_KEY')
    if not api_key:
        raise ValueError("Chave API não configurada!")
    
    return openai.OpenAI(
        api_key=api_key,
        base_url="https://chat.maritaca.ai/api"
    )

def extrair_resposta(texto: str) -> Optional[str]:
    """Extrai resposta do modelo - versão melhorada"""
    if not texto:
        return None
    
    import re
    texto_upper = texto.upper().strip()
    
    # Padrões explícitos (prioridade alta)
    padroes_explicitos = [
        r'RESPOSTA:\s*([A-E])',
        r'ALTERNATIVA\s*([A-E])',
        r'GABARITO:\s*([A-E])',
        r'A\s*RESPOSTA\s*É\s*([A-E])',
        r'RESPOSTA\s*CORRETA:\s*([A-E])',
        r'LETRA\s*([A-E])',
    ]
    
    for padrao in padroes_explicitos:
        match = re.search(padrao, texto_upper)
        if match:
            letra = match.group(1)
            if letra in ['A', 'B', 'C', 'D', 'E']:
                return letra
    
    # Procurar por padrões como "A)", "B)", etc no final do texto
    padroes_finais = re.findall(r'\b([A-E])\s*[\)\.:]?\s*$', texto_upper[-200:])
    if padroes_finais:
        letra = padroes_finais[-1]
        if letra in ['A', 'B', 'C', 'D', 'E']:
            return letra
    
    # Último recurso: procurar qualquer letra A-E isolada no final
    # (mas com cuidado para não pegar letras no meio do texto)
    palavras_finais = texto_upper.split()[-10:]  # Últimas 10 palavras
    for palavra in reversed(palavras_finais):
        palavra_limpa = re.sub(r'[^A-E]', '', palavra)
        if palavra_limpa in ['A', 'B', 'C', 'D', 'E']:
            return palavra_limpa
    
    return None

def carregar_questoes_colab(anos=None, apenas_2024=False, incluir_treino=True):
    """
    Carrega questões de TODAS as fontes: ENEM, ITA, IME, FUVEST, UNICAMP
    
    Args:
        anos: Lista de anos ENEM para carregar (ex: [2022, 2023, 2024]). Se None, carrega todos.
        apenas_2024: Se True, carrega apenas ENEM 2024 (180 questões). Se False, carrega todos os anos ENEM.
        incluir_treino: Se True, carrega também ITA, IME, FUVEST, UNICAMP de data/treino/
    
    Returns:
        Dict com questões organizadas por área
    """
    processed_dir = Path('data/processed')
    treino_dir = Path('data/treino')
    
    questoes_por_area = {
        'languages': [],
        'human-sciences': [],
        'natural-sciences': [],
        'mathematics': []
    }
    
    total_carregado = 0
    
    # ===== CARREGAR ENEM =====
    if processed_dir.exists():
        if apenas_2024:
            arquivos_enem = [processed_dir / 'enem_2024_completo.jsonl']
            print("📁 Carregando apenas ENEM 2024 (180 questões)")
        else:
            if anos:
                arquivos_enem = [processed_dir / f'enem_{ano}_completo.jsonl' for ano in anos]
            else:
                arquivos_enem = sorted(processed_dir.glob('enem_*_completo.jsonl'))
            print(f"📁 Carregando ENEM: {len(arquivos_enem)} arquivo(s)")
        
        for arquivo in arquivos_enem:
            if not arquivo.exists():
                continue
            
            ano = arquivo.stem.replace('enem_', '').replace('_completo', '')
            
            try:
                with open(arquivo, 'r', encoding='utf-8') as f:
                    count_arquivo = 0
                    for line in f:
                        if line.strip():
                            questao = json.loads(line)
                            num_str = questao.get('id', '').replace('questao_', '') or questao.get('number', '')
                            
                            try:
                                num = int(num_str)
                                # Classificar por número (ENEM sempre tem 45 por área)
                                if 1 <= num <= 45:
                                    area = 'languages'
                                elif 46 <= num <= 90:
                                    area = 'human-sciences'
                                elif 91 <= num <= 135:
                                    area = 'natural-sciences'
                                elif 136 <= num <= 180:
                                    area = 'mathematics'
                                else:
                                    continue
                                
                                # Adicionar informação
                                questao['number'] = num
                                questao['area'] = area
                                questao['ano'] = ano
                                questao['exame'] = 'ENEM'
                                
                                questoes_por_area[area].append(questao)
                                count_arquivo += 1
                                total_carregado += 1
                            except (ValueError, TypeError):
                                continue
                    
                    if count_arquivo > 0:
                        print(f"   ✅ ENEM {ano}: {count_arquivo} questões")
            except Exception as e:
                print(f"   ⚠️  Erro ao carregar {arquivo.name}: {str(e)[:50]}")
                continue
    else:
        print(f"⚠️  Diretório ENEM não encontrado: {processed_dir}")
    
    # ===== CARREGAR ITA, IME, FUVEST, UNICAMP =====
    if incluir_treino and treino_dir.exists():
        print(f"\n📁 Carregando questões de treino (ITA, IME, FUVEST, UNICAMP)...")
        
        arquivos_treino = [
            ('FUVEST', treino_dir / 'treino_fuvest.jsonl'),
            ('ITA', treino_dir / 'treino_ita.jsonl'),
            ('IME', treino_dir / 'treino_ime.jsonl'),
            ('UNICAMP', treino_dir / 'treino_unicamp.jsonl'),
        ]
        
        for exame_nome, arquivo in arquivos_treino:
            if not arquivo.exists():
                continue
            
            try:
                with open(arquivo, 'r', encoding='utf-8') as f:
                    count_arquivo = 0
                    for line in f:
                        if line.strip():
                            questao = json.loads(line)
                            
                            # Questões de treino já têm 'area' definida
                            area = questao.get('area', '').lower()
                            
                            # Mapear áreas
                            if area in ['languages', 'human-sciences', 'natural-sciences', 'mathematics']:
                                # Adicionar informação do exame
                                questao['exame'] = exame_nome
                                questao['exam_type'] = questao.get('exam_type', exame_nome.lower())
                                
                                questoes_por_area[area].append(questao)
                                count_arquivo += 1
                                total_carregado += 1
                    
                    if count_arquivo > 0:
                        print(f"   ✅ {exame_nome}: {count_arquivo} questões")
            except Exception as e:
                print(f"   ⚠️  Erro ao carregar {arquivo.name}: {str(e)[:50]}")
                continue
    elif incluir_treino:
        print(f"⚠️  Diretório de treino não encontrado: {treino_dir}")
    
    # ===== ESTATÍSTICAS FINAIS =====
    print(f"\n📊 Total carregado: {total_carregado} questões")
    print(f"\n📊 Por área:")
    for area, questoes in questoes_por_area.items():
        print(f"   {area}: {len(questoes)} questões")
    
    # Estatísticas por exame
    exames_count = {}
    for area_questoes in questoes_por_area.values():
        for q in area_questoes:
            exame = q.get('exame', 'Desconhecido')
            exames_count[exame] = exames_count.get(exame, 0) + 1
    
    if exames_count:
        print(f"\n📊 Por exame:")
        for exame, count in sorted(exames_count.items()):
            print(f"   {exame}: {count} questões")
    
    return questoes_por_area

print("✅ Funções auxiliares carregadas!")


✅ Funções auxiliares carregadas!


In [49]:
# 🔧 PREPARAR DIRETÓRIOS DE DADOS
# Criar diretórios se não existirem
from pathlib import Path
import os

data_dirs = [
    Path('data/processed'),
    Path('data/treino'),
    Path('results')
]

print("📁 Preparando diretórios de dados...")
for dir_path in data_dirs:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"   ✅ {dir_path}")

# Verificar se há dados
processed_dir = Path('data/processed')
treino_dir = Path('data/treino')

enem_files = list(processed_dir.glob('enem_*.jsonl')) if processed_dir.exists() else []
treino_files = list(treino_dir.glob('treino_*.jsonl')) if treino_dir.exists() else []

print(f"\n📊 Status dos dados:")
print(f"   ENEM: {len(enem_files)} arquivo(s) encontrado(s)")
print(f"   Treino: {len(treino_files)} arquivo(s) encontrado(s)")

if len(enem_files) == 0 and len(treino_files) == 0:
    print("\n⚠️  NENHUM DADO ENCONTRADO!")
    print("\n💡 OPÇÕES PARA CARREGAR DADOS:")
    print("\n   1. FAZER UPLOAD MANUAL (Recomendado):")
    print("      - Clique no ícone de pasta 📁 no Colab")
    print("      - Faça upload dos arquivos:")
    print("        * data/processed/enem_2024_completo.jsonl")
    print("        * data/treino/treino_*.jsonl (opcional)")
    print("\n   2. BAIXAR DADOS DO GITHUB (se disponíveis):")
    print("      - Execute a célula abaixo para tentar baixar")
    print("\n   3. USAR DADOS DE EXEMPLO:")
    print("      - O sistema pode funcionar com dados mínimos")
    print("      - Mas acurácia será limitada")
    
    print("\n💡 SOLUÇÃO: Fazer upload do DATASET COMPLETO")
    print("\n   Opção 1: Upload via código (próxima célula) - RECOMENDADO")
    print("      - Execute a célula 10")
    print("      - Selecione TODOS os arquivos de uma vez:")
    print("        * ENEM 2009-2025 (17 arquivos)")
    print("        * Treino: FUVEST, ITA, IME, UNICAMP (4 arquivos)")
    print("\n   Opção 2: Upload manual via interface do Colab")
    print("      - Clique no ícone 📁 (pasta) na barra lateral")
    print("      - Navegue até data/processed/ e faça upload de todos os ENEM")
    print("      - Navegue até data/treino/ e faça upload de todos os treinos")
    print("\n   ⚠️  IMPORTANTE: Os dados não estão no GitHub (estão no .gitignore)")
    print("      Você precisa fazer upload do dataset completo dos seus arquivos locais")
else:
    print("\n✅ Dados encontrados! Pronto para carregar questões.")


📁 Preparando diretórios de dados...
   ✅ data/processed
   ✅ data/treino
   ✅ results

📊 Status dos dados:
   ENEM: 21 arquivo(s) encontrado(s)
   Treino: 5 arquivo(s) encontrado(s)

✅ Dados encontrados! Pronto para carregar questões.


In [50]:
# 📤 UPLOAD DE DATASET COMPLETO
# Faça upload de TODOS os arquivos de uma vez!
# Você pode selecionar múltiplos arquivos simultaneamente

from pathlib import Path
import os

# Detectar se está no Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import files
else:
    print("⚠️  Não está no Google Colab!")
    print("   Esta célula só funciona no Colab.")
    print("   Use a interface do Colab (ícone 📁) para fazer upload manualmente.")
    files = None

print("📤 UPLOAD DE DATASET COMPLETO")
print("=" * 70)
print("\n💡 INSTRUÇÕES:")
print("   1. Execute esta célula")
print("   2. Clique em 'Escolher arquivos' e selecione TODOS os arquivos de uma vez:")
print("      - ENEM: enem_2009_completo.jsonl até enem_2025_completo.jsonl (17 arquivos)")
print("      - Treino: treino_fuvest.jsonl, treino_ita.jsonl, treino_ime.jsonl, treino_unicamp.jsonl")
print("   3. Os arquivos serão organizados automaticamente nas pastas corretas")
print("\n📁 DATASET COMPLETO:")
print("   📚 ENEM (2009-2025):")
print("      - enem_2009_completo.jsonl")
print("      - enem_2010_completo.jsonl")
print("      - ... (até 2025)")
print("      - enem_2025_completo.jsonl")
print("   📚 TREINO:")
print("      - treino_fuvest.jsonl")
print("      - treino_ita.jsonl")
print("      - treino_ime.jsonl")
print("      - treino_unicamp.jsonl")
print("\n💡 DICA: Você pode selecionar múltiplos arquivos de uma vez!")
print("   - No Windows/Linux: Ctrl+Clique para selecionar múltiplos")
print("   - No Mac: Cmd+Clique para selecionar múltiplos")
print("\n⚠️  Alternativa: Use a interface do Colab (ícone 📁 na barra lateral)")
print("   - Faça upload diretamente nas pastas data/processed/ e data/treino/")
print("\n" + "=" * 70)

# Criar diretórios se não existirem
Path('data/processed').mkdir(parents=True, exist_ok=True)
Path('data/treino').mkdir(parents=True, exist_ok=True)

if not IN_COLAB:
    print("\n❌ Esta funcionalidade requer Google Colab.")
    print("   Use a interface do Colab (ícone 📁 na barra lateral) para fazer upload.")
else:
    print("\n🔽 Selecione os arquivos para upload...")
    print("   (Pressione Ctrl+C se preferir fazer upload manual)")

try:
    if not IN_COLAB:
        uploaded = {}
    else:
        uploaded = files.upload()
    
    if uploaded:
        print(f"\n✅ {len(uploaded)} arquivo(s) recebido(s)!")
        
        for filename, content in uploaded.items():
            # Determinar destino baseado no nome
            if 'enem_' in filename and filename.endswith('.jsonl'):
                dest = Path('data/processed') / filename
            elif 'treino_' in filename and filename.endswith('.jsonl'):
                dest = Path('data/treino') / filename
            else:
                # Tentar adivinhar
                if 'processed' in filename or 'enem' in filename.lower():
                    dest = Path('data/processed') / filename
                else:
                    dest = Path('data/treino') / filename
            
            # Salvar arquivo
            dest.parent.mkdir(parents=True, exist_ok=True)
            with open(dest, 'wb') as f:
                f.write(content)
            
            size_kb = len(content) / 1024
            print(f"   ✅ {filename} → {dest} ({size_kb:.1f} KB)")
        
        print("\n🎉 Upload concluído! Execute a célula anterior novamente para verificar.")
    else:
        print("\n⚠️  Nenhum arquivo selecionado.")
        print("   Use a interface do Colab (ícone 📁) para fazer upload manualmente.")
        
except KeyboardInterrupt:
    print("\n\n⚠️  Upload cancelado.")
    print("   Use a interface do Colab (ícone 📁) para fazer upload manualmente.")
except Exception as e:
    print(f"\n❌ Erro durante upload: {str(e)}")
    print("   Use a interface do Colab (ícone 📁) para fazer upload manualmente.")


⚠️  Não está no Google Colab!
   Esta célula só funciona no Colab.
   Use a interface do Colab (ícone 📁) para fazer upload manualmente.
📤 UPLOAD DE DATASET COMPLETO

💡 INSTRUÇÕES:
   1. Execute esta célula
   2. Clique em 'Escolher arquivos' e selecione TODOS os arquivos de uma vez:
      - ENEM: enem_2009_completo.jsonl até enem_2025_completo.jsonl (17 arquivos)
      - Treino: treino_fuvest.jsonl, treino_ita.jsonl, treino_ime.jsonl, treino_unicamp.jsonl
   3. Os arquivos serão organizados automaticamente nas pastas corretas

📁 DATASET COMPLETO:
   📚 ENEM (2009-2025):
      - enem_2009_completo.jsonl
      - enem_2010_completo.jsonl
      - ... (até 2025)
      - enem_2025_completo.jsonl
   📚 TREINO:
      - treino_fuvest.jsonl
      - treino_ita.jsonl
      - treino_ime.jsonl
      - treino_unicamp.jsonl

💡 DICA: Você pode selecionar múltiplos arquivos de uma vez!
   - No Windows/Linux: Ctrl+Clique para selecionar múltiplos
   - No Mac: Cmd+Clique para selecionar múltiplos

⚠️  Alter

In [51]:
# Configurar
client = configurar_api_colab()

# CARREGAR QUESTÕES
# Verificar se a função tem os parâmetros novos
import inspect
try:
    sig = inspect.signature(carregar_questoes_colab)
    params = list(sig.parameters.keys())
    tem_parametro_novo = 'apenas_2024' in params
    
    if tem_parametro_novo:
        print("✅ Função atualizada detectada")
        # Opção 1: Apenas ENEM 2024 (180 questões) - mais rápido para testes
        # questoes_por_area = carregar_questoes_colab(apenas_2024=True)
        
        # Opção 2: Todos os anos disponíveis (dataset completo) - RECOMENDADO
        questoes_por_area = carregar_questoes_colab(apenas_2024=False)
        
        # Opção 3: Anos específicos (ex: últimos 3 anos)
        # questoes_por_area = carregar_questoes_colab(anos=[2022, 2023, 2024])
    else:
        print("⚠️  Função antiga detectada - usando versão compatível")
        print("   (Execute a célula 8 novamente para usar versão completa)")
        # Versão compatível: chamar sem parâmetros (carrega apenas 2024)
        questoes_por_area = carregar_questoes_colab()
        
except NameError:
    print("❌ ERRO: Função carregar_questoes_colab não encontrada!")
    print("   Execute a célula 8 primeiro!")
    raise
except TypeError as e:
    # Se der erro de parâmetro, tentar sem parâmetros
    print("⚠️  Tentando versão compatível...")
    questoes_por_area = carregar_questoes_colab()

area_names = {
    'languages': 'Linguagens',
    'human-sciences': 'Humanas',
    'natural-sciences': 'Natureza',
    'mathematics': 'Matemática'
}

total = sum(len(q) for q in questoes_por_area.values())
print(f"\n✅ Total: {total} questões carregadas")
for area, name in area_names.items():
    count = len(questoes_por_area.get(area, []))
    print(f"   {name}: {count} questões")


✅ Função atualizada detectada
📁 Carregando ENEM: 17 arquivo(s)
   ✅ ENEM 2024: 180 questões

📁 Carregando questões de treino (ITA, IME, FUVEST, UNICAMP)...
   ✅ FUVEST: 1282 questões
   ✅ ITA: 714 questões
   ✅ IME: 147 questões
   ✅ UNICAMP: 708 questões

📊 Total carregado: 3031 questões

📊 Por área:
   languages: 562 questões
   human-sciences: 721 questões
   natural-sciences: 1224 questões
   mathematics: 524 questões

📊 Por exame:
   ENEM: 180 questões
   FUVEST: 1282 questões
   IME: 147 questões
   ITA: 714 questões
   UNICAMP: 708 questões

✅ Total: 3031 questões carregadas
   Linguagens: 562 questões
   Humanas: 721 questões
   Natureza: 1224 questões
   Matemática: 524 questões


In [52]:
# Configuração do teste - APROVEITANDO PODER DE PROCESSAMENTO!
# Opções:
# - TESTE RÁPIDO: QUESTOES_POR_AREA = 10, N_PASSAGENS = 3
# - TESTE MÉDIO: QUESTOES_POR_AREA = 45, N_PASSAGENS = 5 (recomendado)
# - TESTE COMPLETO: QUESTOES_POR_AREA = None (todas), N_PASSAGENS = 7 (máxima qualidade)

QUESTOES_POR_AREA = 45  # 45 = todas as questões de uma área do ENEM (ou None para todas)
N_PASSAGENS = 5  # Self-consistency (3=rápido, 5=balanceado, 7=máxima qualidade)

print(f"📊 Configuração - Aproveitando GPU do Colab:")
print(f"   Questões por área: {QUESTOES_POR_AREA if QUESTOES_POR_AREA else 'TODAS'}")
print(f"   Passagens (self-consistency): {N_PASSAGENS}")
if QUESTOES_POR_AREA:
    total_questoes = QUESTOES_POR_AREA * 4
    total_api_calls = total_questoes * N_PASSAGENS
    tempo_estimado = (total_api_calls * 0.5) / 60  # ~0.5s por chamada
    print(f"   Total de questões: {total_questoes}")
    print(f"   Total de chamadas API: {total_api_calls}")
    print(f"   Tempo estimado: ~{tempo_estimado:.1f} minutos")
else:
    print(f"   Total de questões: TODAS disponíveis")
    print(f"   ⚠️  Isso pode levar muito tempo!")


📊 Configuração - Aproveitando GPU do Colab:
   Questões por área: 45
   Passagens (self-consistency): 5
   Total de questões: 180
   Total de chamadas API: 900
   Tempo estimado: ~7.5 minutos


In [53]:
# Função completa de avaliação
def construir_prompt_final_colab(questao, prompts_module, fewshots_module, figuras_module, 
                                simples_module, natureza_module, prompt_nat_module):
    """Constrói prompt final com todas as melhorias"""
    num = questao.get('number', 0)
    area = questao.get('area', '')
    
    # Obter TRI (só funciona para Matemática 136-180)
    tri_info = prompts_module.obter_info_tri(num)
    tri_value = tri_info.get('TRI', 0)
    nivel = prompts_module.classificar_por_tri(tri_value) if tri_value > 0 else 'medio'
    tema = tri_info.get('Tema', 'N/A')
    
    # Prompt adaptativo baseado em TRI (se disponível) ou área
    if tri_value > 0:
        # Temos TRI - usar prompt adaptativo
        prompt_base = prompts_module.selecionar_prompt_por_tri(tri_value)
    else:
        # Sem TRI - criar prompt específico por área
        if area == 'languages':
            prompt_base = """Você é um especialista em questões de Linguagens do ENEM.

Esta questão envolve interpretação de texto, gramática, literatura ou artes.

📋 METODOLOGIA PARA LINGUAGENS:

1. LEITURA ATENTA
   - Leia o texto/contexto com cuidado
   - Identifique o gênero textual
   - Observe elementos de coesão e coerência

2. ANÁLISE DA PERGUNTA
   - O que exatamente está sendo perguntado?
   - É sobre interpretação, gramática, literatura ou artes?
   - Identifique palavras-chave na pergunta

3. ANÁLISE DAS ALTERNATIVAS
   - Leia todas as alternativas antes de escolher
   - Elimine alternativas claramente incorretas
   - Compare com o texto/contexto fornecido

4. VALIDAÇÃO
   - Sua resposta está fundamentada no texto?
   - Faz sentido no contexto?
   - Responde exatamente o que foi perguntado?

Agora, resolva a questão abaixo:

"""
        elif area == 'human-sciences':
            prompt_base = """Você é um especialista em questões de Ciências Humanas do ENEM.

Esta questão envolve História, Geografia, Filosofia ou Sociologia.

📋 METODOLOGIA PARA CIÊNCIAS HUMANAS:

1. CONTEXTUALIZAÇÃO
   - Identifique o período histórico ou contexto geográfico
   - Relacione com conceitos de História, Geografia, Filosofia ou Sociologia
   - Observe dados, mapas ou gráficos fornecidos

2. ANÁLISE DA PERGUNTA
   - O que está sendo perguntado?
   - Qual área do conhecimento (História, Geografia, Filosofia, Sociologia)?
   - Identifique conceitos-chave

3. RELAÇÃO COM O CONTEXTO
   - Relacione a pergunta com o contexto fornecido
   - Use conhecimentos históricos/geográficos relevantes
   - Considere múltiplas perspectivas quando aplicável

4. ANÁLISE DAS ALTERNATIVAS
   - Elimine alternativas anacrônicas ou geograficamente incorretas
   - Compare com o contexto fornecido
   - Verifique se a resposta está fundamentada

Agora, resolva a questão abaixo:

"""
        elif area == 'natural-sciences':
            prompt_base = """Você é um especialista em questões de Ciências da Natureza do ENEM.

Esta questão envolve Física, Química ou Biologia.

📋 METODOLOGIA PARA CIÊNCIAS DA NATUREZA:

1. IDENTIFICAÇÃO DO PROBLEMA
   - Leia o contexto e a pergunta
   - Identifique a área (Física, Química ou Biologia)
   - Anote os dados fornecidos

2. CONCEITOS CIENTÍFICOS
   - Identifique os conceitos científicos envolvidos
   - Relacione com fórmulas ou princípios quando aplicável
   - Considere unidades de medida

3. RESOLUÇÃO
   - Resolva passo a passo
   - Mostre cálculos quando necessário
   - Verifique cada etapa

4. VALIDAÇÃO
   - Verifique se a resposta faz sentido cientificamente
   - Confirme unidades e ordens de grandeza
   - Compare com as alternativas

Agora, resolva a questão abaixo:

"""
        else:
            # Matemática sem TRI ou área desconhecida
            prompt_base = """Você é um especialista em questões do ENEM. Resolva passo-a-passo.

📋 METODOLOGIA:

1. Leia o contexto e a pergunta com atenção
2. Identifique o que está sendo pedido
3. Resolva passo a passo
4. Verifique se a resposta faz sentido
5. Escolha a alternativa correta

Agora, resolva a questão abaixo:

"""
    
    # Prompt ultra-simples se fácil
    if tri_value > 0 and tri_value < 650:
        prompt_base = simples_module.aplicar_prompt_ultra_simples(
            prompt_base, questao, tri_value, figuras_module.obter_info_figura
        )
    
    # Few-shots e prompts específicos
    if area == 'natural-sciences' and prompt_nat_module:
        prompt_base = prompt_nat_module.criar_prompt_natureza(prompt_base)
        if nivel == 'medio' and natureza_module:
            fewshots = natureza_module.obter_fewshots_natureza(5)
            for fs in fewshots:
                prompt_base += f"\n\nExemplo:\n{fs['question']}\n{fs['response']}\n"
    elif area == 'mathematics' and nivel == 'medio' and fewshots_module:
        prompt_base = fewshots_module.criar_prompt_com_fewshots(prompt_base, tema, 3)
    
    # Detecção de figuras
    prompt_final = figuras_module.criar_prompt_com_deteccao_figura(prompt_base, questao)
    
    return prompt_final, {'tri': tri_value, 'nivel': nivel, 'tema': tema, 'area': area}

def formatar_questao_colab(questao):
    """Formata questão"""
    texto = ""
    if questao.get('context'):
        texto += f"CONTEXTO:\n{questao['context']}\n\n"
    if questao.get('description'):
        desc = questao['description']
        if isinstance(desc, list) and desc:
            texto += f"DESCRIÇÃO DAS IMAGENS:\n{desc[0]}\n\n"
        elif desc:
            texto += f"DESCRIÇÃO DAS IMAGENS:\n{desc}\n\n"
    texto += f"PERGUNTA:\n{questao.get('question', '')}\n\n"
    texto += "ALTERNATIVAS:\n"
    for i, alt in enumerate(questao.get('alternatives', []), 1):
        letra = chr(64 + i)
        texto += f"{letra}) {alt}\n"
    return texto

def resolver_com_self_consistency_colab(client, questao, n_passagens, prompts_module, 
                                         fewshots_module, figuras_module, simples_module,
                                         natureza_module, prompt_nat_module, usar_paralelo=False):
    """
    Resolve com self-consistency
    
    Args:
        usar_paralelo: Se True, tenta processar passagens em paralelo (requer GPU/threading)
    """
    respostas = []
    
    prompt_final, info = construir_prompt_final_colab(
        questao, prompts_module, fewshots_module, figuras_module,
        simples_module, natureza_module, prompt_nat_module
    )
    questao_formatada = formatar_questao_colab(questao)
    prompt_completo = prompt_final + questao_formatada
    
    # Otimização: reduzir delay se GPU disponível
    delay = 0.1 if usar_paralelo else 0.2
    
    for i in range(n_passagens):
        try:
            response = client.chat.completions.create(
                model="sabia-3",
                messages=[
                    {"role": "system", "content": "Você é um especialista em questões do ENEM."},
                    {"role": "user", "content": prompt_completo}
                ],
                temperature=0.1,
                max_tokens=2000
            )
            
            resposta_texto = response.choices[0].message.content
            resposta_extraida = extrair_resposta(resposta_texto)
            
            if resposta_extraida:
                respostas.append(resposta_extraida)
            
            time.sleep(delay)  # Delay reduzido se GPU disponível
        except Exception as e:
            print(f"      ⚠️  Erro passagem {i+1}: {str(e)[:50]}")
            continue
    
    if not respostas:
        return None, 0.0, info
    
    # Votação majoritária
    contador = Counter(respostas)
    resposta_mais_frequente = contador.most_common(1)[0]
    resposta_final = resposta_mais_frequente[0]
    frequencia = resposta_mais_frequente[1]
    confianca = frequencia / len(respostas)
    
    return resposta_final, confianca, info

print("✅ Funções de avaliação carregadas!")


✅ Funções de avaliação carregadas!


In [55]:
# EXECUTAR AVALIAÇÃO COMPLETA
print("=" * 70)
print("🚀 AVALIAÇÃO COMPLETA - BRAINX")
print("=" * 70)
print()

# Verificar se GPU está disponível para otimização
try:
    import torch
    usar_gpu_otimizado = torch.cuda.is_available()
    if usar_gpu_otimizado:
        print("✅ GPU detectada - usando otimizações")
    else:
        print("⚠️  GPU não detectada - processamento padrão")
except:
    usar_gpu_otimizado = False

stats_por_area = defaultdict(lambda: {'correct': 0, 'total': 0})
resultados = []
total_geral = 0
correct_geral = 0
start_time = time.time()

for area_key, area_name in area_names.items():
    questoes = questoes_por_area.get(area_key, [])
    
    # Limitar se QUESTOES_POR_AREA estiver definido
    if QUESTOES_POR_AREA:
        questoes = questoes[:QUESTOES_POR_AREA]
    
    # Se não há limite, usar todas (mas avisar)
    if not QUESTOES_POR_AREA and len(questoes) > 100:
        print(f"⚠️  {area_name}: {len(questoes)} questões - isso pode levar muito tempo!")
    
    if not questoes:
        print(f"⚠️  Nenhuma questão para {area_name}")
        continue
    
    print(f"\n📚 {area_name} ({len(questoes)} questões)")
    print("-" * 70)
    
    # Otimização: reduzir delay entre questões se GPU disponível
    delay_entre_questoes = 0.1 if usar_gpu_otimizado else 0.3
    
    for q in tqdm(questoes, desc=f"{area_name}"):
        q_num = q.get('number', 0)
        total_geral += 1
        stats_por_area[area_name]['total'] += 1
        
        resposta_final, confianca, info = resolver_com_self_consistency_colab(
            client, q, N_PASSAGENS, prompts_module, fewshots_module,
            figuras_module, simples_module, natureza_module, prompt_nat_module,
            usar_paralelo=usar_gpu_otimizado
        )
        
        # Normalizar label (garantir maiúscula)
        correct_answer_raw = q.get('label', '') or q.get('answer', '') or q.get('gabarito', '')
        correct_answer = str(correct_answer_raw).upper().strip()
        
        # Validar que é uma alternativa válida
        if correct_answer not in ['A', 'B', 'C', 'D', 'E']:
            print(f"  Q{q_num}: ⚠️  Gabarito inválido: '{correct_answer_raw}'")
            correct_answer = ''
        
        # Comparar (normalizar resposta também)
        resposta_normalizada = str(resposta_final).upper().strip() if resposta_final else ''
        is_correct = (resposta_normalizada == correct_answer) if (resposta_normalizada and correct_answer) else False
        
        if is_correct:
            correct_geral += 1
            stats_por_area[area_name]['correct'] += 1
            print(f"  Q{q_num}: ✅ ({resposta_final}, conf: {confianca:.0%})")
        else:
            print(f"  Q{q_num}: ❌ ({resposta_final} vs {correct_answer}, conf: {confianca:.0%})")
        
        resultados.append({
            'area': area_name,
            'numero': q_num,
            'resposta': resposta_final,
            'gabarito': correct_answer,
            'correto': is_correct,
            'confianca': confianca
        })
        
        time.sleep(delay_entre_questoes)  # Delay otimizado

elapsed_time = time.time() - start_time
accuracy_geral = correct_geral / total_geral if total_geral > 0 else 0

print("\n" + "=" * 70)
print("📊 RESULTADOS FINAIS")
print("=" * 70)
print()
print(f"🎯 Acurácia Geral: {accuracy_geral:.2%} ({correct_geral}/{total_geral})")
print(f"⏱️  Tempo: {elapsed_time:.1f}s ({elapsed_time/total_geral:.1f}s por questão)")
print()

print("📊 Por Área:")
print("-" * 50)
for area_name in ['Linguagens', 'Humanas', 'Natureza', 'Matemática']:
    stats = stats_por_area[area_name]
    if stats['total'] > 0:
        acc = stats['correct'] / stats['total']
        print(f"{area_name:<15} {stats['correct']:>3}/{stats['total']:<3} = {acc:>5.1f}%")

print()
print("📈 Comparação:")
print("-" * 50)
print(f"BrainX (Teste):     {accuracy_geral:.2%}")
print(f"BrainX (Atual):     86.59%")
print(f"GPT-4o (Paper):     93.85%")
print(f"Gap para GPT-4o:    {93.85 - accuracy_geral*100:+.2f} pontos")

if accuracy_geral >= 0.94:
    print("\n🎉 PARABÉNS! BrainX superou GPT-4o!")
elif accuracy_geral >= 0.90:
    print("\n✅ Excelente! Muito próximo do GPT-4o!")
elif accuracy_geral >= 0.87:
    print("\n✅ Bom resultado! Melhorias funcionando!")

# Salvar resultados
from datetime import datetime
output_file = Path(f"results/avaliacao_colab_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
output_file.parent.mkdir(exist_ok=True)

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump({
        'total': total_geral,
        'correct': correct_geral,
        'accuracy': accuracy_geral,
        'stats_por_area': dict(stats_por_area),
        'resultados': resultados,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2, ensure_ascii=False)

print(f"\n💾 Resultados salvos em: {output_file}")


🚀 AVALIAÇÃO COMPLETA - BRAINX

⚠️  GPU não detectada - processamento padrão

📚 Linguagens (45 questões)
----------------------------------------------------------------------


Linguagens:   0%|          | 0/45 [00:00<?, ?it/s]


KeyboardInterrupt: 